In [2]:
from PIL import Image
import numpy as np
import cv2
import dlib
import os

def ela(image_path, quality=95):
    """
    Performs Error Level Analysis (ELA) on the input image.
    
    Args:
        image_path (str): Path to the input image.
        quality (int): JPEG quality level for resaving (default: 95).
    
    Returns:
        np.ndarray: Grayscale ELA image.
    """
    original = Image.open(image_path).convert('RGB')
    resaved_path = 'resaved.jpg'
    original.save(resaved_path, 'JPEG', quality=quality)
    resaved = Image.open(resaved_path).convert('RGB')
    diff = np.array(original) - np.array(resaved)
    ela = np.max(np.abs(diff), axis=2)
    ela = (ela / ela.max() * 255).astype(np.uint8)
    return ela

def detect_landmarks(image_path):
    """
    Detects 68 facial landmarks using dlib.
    
    Args:
        image_path (str): Path to the input image.
    
    Returns:
        list: List of (x, y) coordinates of facial landmarks.
    """
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("/home/subrahmanya/projects/UnFake/shape_predictor_68_face_landmarks.dat")
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    landmarks = []
    for face in faces:
        shape = predictor(gray, face)
        for n in range(0, 68):
            x = shape.part(n).x
            y = shape.part(n).y
            landmarks.append((x, y))
    return landmarks

def advanced_feature_extraction(image_path, ela_threshold=50, landmark_radius=5):
    """
    Extracts and highlights potentially tampered facial features using ELA and landmark detection.
    
    Args:
        image_path (str): Path to the input image.
        ela_threshold (int): Threshold for high ELA values (default: 50).
        landmark_radius (int): Radius around landmarks to consider (default: 5 pixels).
    
    Returns:
        None: Saves the highlighted image as 'tampered_features.png'.
    """
    # Compute ELA
    ela_img = ela(image_path)
    
    # Detect landmarks
    landmarks = detect_landmarks(image_path)
    
    # Create a mask for high ELA regions
    high_ela_mask = ela_img > ela_threshold
    
    # Create a mask for landmarks (small circles around each landmark)
    img = cv2.imread(image_path)
    landmark_mask = np.zeros_like(ela_img)
    for (x, y) in landmarks:
        cv2.circle(landmark_mask, (x, y), landmark_radius, 255, -1)
    
    # Find overlapping regions (high ELA and near landmarks)
    overlap = cv2.bitwise_and(high_ela_mask.astype(np.uint8), landmark_mask)
    
    # Highlight overlapping regions on the original image
    img_highlight = img.copy()
    img_highlight[overlap > 0] = (0, 0, 255)  # Red color for tampered landmarks
    
    # Save the result
    cv2.imwrite('tampered_features.png', img_highlight)

# Example usage
# Ensure 'shape_predictor_68_face_landmarks.dat' is in the working directory
advanced_feature_extraction('/home/subrahmanya/projects/UnFake/images/oldman-face.jpeg')